In [ ]:
# I don't have tensorflow and wasn't using it so commented out to make the logreg and svm models
# import tensorflow as tf
# from tensorflow.keras import datasets, layers, models
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from PIL import Image

# sklearn utilities
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# sklearn models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA

In [ ]:
# Adds images to array
images = np.zeros((2000, 250, 250, 3)) # I changed this to be the actual dimensions that np.asarray will copy into
j = 0
for i in range(1522): # Our ranges were off by 1 so I fixed that
  try:
    image_name = 'F_' + str(i) + '.jpg'
    path = "Fire/" + image_name
    image = (Image.open(path))
    img = np.asarray(image)
    images[j] = np.copy(img)
    j = j+1
  except:
    continue

In [ ]:
for i in range(1011): # Range was off by 1 like above
  try:
    image_name = 'NF_' + str(i) + '.jpg'
    path = 'Non_Fire/' + image_name
    image = (Image.open(path))
    img = np.asarray(image)
    images[j] = np.copy(img)
    j = j+1
  except:
    continue

In [ ]:
targets = np.zeros(2000)
for i in range(1000):
  targets[i] = 1

In [ ]:
# For genuine results don't set a random_state
x_train, x_test, y_train, y_test = train_test_split(images, targets, test_size=.25, random_state=0)

In [ ]:
# Confirm shape of data and targets
print("Training x:", x_train.shape,"y:", y_train.shape)
print("Testing x:", x_test.shape,"y:", y_test.shape)

In [ ]:
# Reduce x to 2d arrays for logreg
x_train_flat = x_train.reshape((1500, 187500))
x_test_flat = x_test.reshape((500, 187500))

In [ ]:
logreg_model = LogisticRegression()
logreg_model.fit(x_train_flat, y_train)
logreg_results = logreg_model.predict(x_test_flat)

In [ ]:
logreg_conf_matrix = confusion_matrix(y_test, logreg_results)
pd.DataFrame(logreg_conf_matrix, columns=[0, 1], index=[0,1])

In [ ]:
print(classification_report(y_test, logreg_results))

In [ ]:
svm_model = SVC(kernel='linear')
svm_model.fit(x_train_flat, y_train)
svm_results = svm_model.predict(x_test_flat)

In [ ]:
svm_conf_matrix = confusion_matrix(y_test, svm_results)
pd.DataFrame(svm_conf_matrix, columns=[0, 1], index=[0,1])

In [ ]:
print(classification_report(y_test, svm_results))

# Below here is using DataFrame and Convolutional Neural Network (CNN)

In [ ]:
# This will make the images array 2d so it can be better converted into a dataframe and csv
images = images.reshape((2000, 187500))
df = pd.DataFrame(images)
df['target'] = targets

In [ ]:
# Shuffles df so we can split into test/train
df = df.sample(frac = 1)
df.head()

In [ ]:
train_data = df[:1500]
test_data = df[1500:]

In [ ]:
train_images = np.asarray(train_data)
train_targets = train_images[: ,-1]
train_images_minus_targets = np.delete(train_images, -1, axis=1)

test_images = np.asarray(test_data)
test_targets = test_images[: ,-1]
test_images_minus_targets = np.delete(test_images, -1, axis=1)

In [ ]:
train_images_minus_targets = np.reshape(train_images_minus_targets, (1500, 250, 250, 3))
test_images_minus_targets = np.reshape(test_images_minus_targets, (500, 250, 250, 3))

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(250, (3, 3), activation='relu', input_shape=(250, 250, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(500, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(500, (3, 3), activation='relu'))

In [ ]:
model.summary()

In [ ]:
model.add(layers.Flatten())
model.add(layers.Dense(500, activation='relu'))
model.add(layers.Dense(2))

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(train_images_minus_targets, train_targets, epochs=10, 
                    validation_data=(test_images_minus_targets, test_targets))